In [17]:
pip install PyPDF2

SyntaxError: invalid syntax (<ipython-input-17-d76fff1975c8>, line 1)

In [2]:
import os
import PyPDF2

source_folder = 'resume_dataset/pdf_format'
combined_txt_path = 'resume_dataset/pdf_format/combined_resumes.txt'

# Create a list to store text from all PDF files
all_pdf_text = []

# Loop through PDF files in the source folder
for filename in os.listdir(source_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(source_folder, filename)

        # Extract text from PDF
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            pdf_text = ' '.join(page.extract_text() for page in pdf_reader.pages)
            all_pdf_text.append(pdf_text)

# Combine all PDF text into a single string
combined_text = '\n\n'.join(all_pdf_text)  # Adding double newline separator

# Save combined text to a single TXT file
with open(combined_txt_path, 'w', encoding='utf-8') as combined_txt_file:
    combined_txt_file.write(combined_text)

In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

# Check the installed version of spaCy
spacy.__version__

# Check GPU information
!nvidia-smi

Fri Mar  1 20:10:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# Load the annotated data from a JSON file
cv_data = json.load(open('annotations.json','r'))

# Display the number of items in the dataset
len(cv_data)

# Display the first item in the dataset
cv_data[0]

['Meta builds technologies and services that enable people to connect with each other, build communities and grow businesses. These Terms govern your use of Facebook, \r',
 {'entities': [[0, 4, 'ORG'], [155, 163, 'ORG']]}]

In [5]:
# Initialize spaCy configuration files by copying from base_config to config.cfg
# !python -m spacy init fill-config /"your_file_path"/base_config.cfg /"your_file_path"/config/config.cfg
!python -m spacy init fill-config Custom_NER/base_config.cfg Custom_NER/config/config.cfg

✔ Auto-filled config with all values
✔ Saved config
Custom_NER/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [11]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.2)

# Display the number of items in the training and testing sets
len(train), len(test)

# Open a file to log errors during annotation processing
file = open('Custom_NER/trained_models/train_file.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('Custom_NER/trained_models/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('Custom_NER/trained_models/test_data.spacy')

# Close the error log file
file.close()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
100%|██████████| 59/59 [00:00<00:00, 2126.98it/s]


In [18]:
!pip install spacy-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 36.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [20]:
# Train a spaCy NER model using the provided configuration and data
!python -m spacy train Custom_NER/config/config.cfg  --output Custom_NER/trained_models/output  --paths.train Custom_NER/trained_models/train_data.spacy  --paths.dev Custom_NER/trained_models/test_data.spacy --gpu-id 0

ℹ Saving to output directory: Custom_NER/trained_models/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 124kB/s]
config.json: 100% 481/481 [00:00<00:00, 2.67MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 3.56MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 35.3MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 38.0MB/s]
model.safetensors: 100% 499M/499M [00:01<00:00, 296MB/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  